In [1]:
import os
import cv2
import time
import mtcnn
import numpy as np
from utils import *
import face_recognition
from scipy import spatial
from skimage import feature
import matplotlib.pyplot as plt

In [8]:
aug = True
if aug:
    path = 'datasets/MTCNN/augmented'
    file_encodings = 'encodings_aug_lbp.npy'
    predict_save_name = 'predicted_compare_aug_lbp.jpg'
    file_names = 'names_lbp.npy'
else:
    path = 'datasets/single'
    file_encodings = 'encodings.npy'
    predict_save_name = 'predicted_compare.jpg'
    file_names = 'names.npy'

threshold = 0.95

In [9]:
class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        self.numPoints = numPoints
        self.radius = radius

    def describe(self, image, eps=1e-7):
        lbp = feature.local_binary_pattern(
            image, self.numPoints, self.radius, method="uniform").astype("uint8")

        hist = cv2.calcHist([lbp], [0], None, [
                            self.numPoints+2], [0, self.numPoints+2]).flatten()
        #hist = np.histogram(lbp.ravel(), bins=range(0,self.numPoints+3), range=(0,self.numPoints+2))[0]

        hist = hist.astype("float")
        hist /= (hist.sum()+eps)

        return hist

In [10]:
desc = LocalBinaryPatterns(24, 3)

In [20]:

def TIME(f):
    def wrapper(*args):
        start_time = time.time()
        res = f(*args)
        end_time = time.time()
        print('Run-time: %fs\n' % (end_time-start_time))
        return res
    return wrapper


@TIME
def findEncodings():
    encodings, names = [], []
    for person in os.listdir(path):
        if person != '.DS_Store':
            image_dir = path + '/' + person
            pix = os.listdir(image_dir)
            if not os.path.isdir(image_dir):
                os.mkdir(image_dir)
            # Loop through each training image for the current person
            for person_img in pix:
                if person_img != '.DS_Store':
                    # Get the face encodings for the face in each image file
                    img = cv2.imread(image_dir + "/" + person_img)
                    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    # gray = np.array(gray, dtype="float") / 255.0
                    feature = desc.describe(gray)
                    # face = face_recognition.load_image_file(
                    #     image_dir + "/" + person_img)

                    # face_enc = face_recognition.face_encodings(face)[0]
                    # Add face encoding for current image with corresponding label (name) to the training data
                    encodings.append(feature)
                    names.append(person)
    print('\nNumber of classes: %d\n' % len(names))
    with open(file_encodings, 'wb') as f:
        np.save(f, encodings)
    with open(file_names, 'wb') as f:
        np.save(f, names)
    print('Encodings Complete')
    return np.array(encodings), np.array(names)


@TIME
def importFaceEncoded(filename):
    # with open(filename, 'rb') as f:
    encodeListKnown = np.load(filename)
    print(filename + ' has imported!')
    return encodeListKnown


def face_distance(face_encodings, face_to_compare):
    if len(face_encodings) == 0:
        return np.empty((0))
    # normed = np.linalg.norm(face_encodings - face_to_compare, axis=1)
    # return normed.T * normed
    #----
    arr = []
    
    for face_encoding in face_encodings:
        d = 0.5*np.sum(((face_encoding-face_to_compare)**2)/(face_encoding+face_to_compare+1e-10))
        arr.append(d)
        # arr.append(spatial.distance.cosine(face_encoding, face_to_compare))

    return np.array(arr)


@TIME
def predict():
    filename = 'validation/four.jpg'
    img = cv2.imread(filename)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detect_faces(img)
    face_images = crop_face(filename, img, faces)
    labels, confs = [], []
    for face in face_images:
        feature = desc.describe(face)
        d = face_distance(encodings, feature)
        matchIndex = np.argmin(d)
        labels.append(names[matchIndex])
        print(1-d[matchIndex])
        confs.append(1-d[matchIndex])
    print(labels, confs)
    draw_facebox(img, faces, labels, confs, save_img=True)
    # img = draw_box(img, face_encodings, face_locations)
    # plt.imsave(predict_save_name, img)
    # print('Predcited')

In [21]:
replace = False
if not os.path.isfile(file_encodings) or replace:
    encodings, names = findEncodings()
else:
    encodings = importFaceEncoded(file_encodings)
    names = importFaceEncoded(file_names)
    # mapping = importFaceEncoded(dict_cosine)


Number of classes: 2842

Encodings Complete
Run-time: 7.842286s



In [22]:
predict()

Number of face decected: 5
Done: 0.999824 validation/four_0.jpg
Done: 0.999505 validation/four_1.jpg
Done: 0.998978 validation/four_2.jpg
Done: 0.995868 validation/four_3.jpg
Confidence is lower than Threshold (0.96) | 0.886772 validation/four_4.jpg
0.9980938376004888
0.999634903462333
0.9992204521606343
0.9965713642919647
['B1605247', 'B1509922', 'B1709632', 'B1709632'] [0.9980938376004888, 0.999634903462333, 0.9992204521606343, 0.9965713642919647]
Run-time: 0.770267s

